In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the data
data = pd.read_csv('../Data/selected_data/merged_data.csv')
data.info()

C:\Users\kaush\AppData\Local\Temp\ipykernel_25024\1736518648.py:6: DtypeWarning: Columns (33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../Data/selected_data/merged_data.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217242 entries, 0 to 217241
Data columns (total 35 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Unnamed: 0           217242 non-null  int64  
 1   batter               217242 non-null  object 
 2   bowler               217242 non-null  object 
 3   non_striker          217242 non-null  object 
 4   runs_by_bat          217242 non-null  int64  
 5   extra_runs           217242 non-null  int64  
 6   total_runs_delivery  217242 non-null  int64  
 7   current_team_total   217242 non-null  int64  
 8   runs_remain          103118 non-null  float64
 9   batter_runs          217242 non-null  int64  
 10  balls_faced          217242 non-null  int64  
 11  wickets_fallen       217242 non-null  int64  
 12  extra_type           217242 non-null  object 
 13  delivery             217242 non-null  float64
 14  over                 217242 non-null  int64  
 15  wicket_type      

Selecting Player "KM Mendis"

In [3]:
sl_batsmen = data[data['batting_team'] == "Sri Lanka"]['batter'].unique()
sl_batsmen

array(['N Dickwella', 'WU Tharanga', 'EMDY Munaweera', 'DAS Gunaratne',
       'TAM Siriwardana', 'CK Kapugedera', 'S Prasanna', 'BKG Mendis',
       'KMDN Kulasekara', 'SL Malinga', 'MD Shanaka', 'JRMVB Sanjaya',
       'MDKJ Perera', 'NLTC Perera', 'AD Mathews', 'SMA Priyanjan',
       'I Udana', 'MD Gunathilaka', 'S Samarawickrama', 'ML Udawatte',
       'SS Pathirana', 'PC de Silva', 'A Dananjaya', 'PVD Chameera',
       'MVT Fernando', 'N Pradeep', 'LD Chandimal', 'BMAJ Mendis',
       'RAS Lakmal', 'DM de Silva', 'PHKD Mendis', 'MA Aponso',
       'PADLR Sandakan', 'WIA Fernando', 'AK Perera', 'CAK Rajitha',
       'CBRLS Kumara', 'PBB Rajapaksa', 'BOP Fernando', 'PWH de Silva',
       'GSNFG Jayasuriya', 'LD Madushanka', 'M Bhanuka', 'B Fernando',
       'P Nissanka', 'KNA Bandara', 'KIC Asalanka', 'C Karunaratne',
       'RTM Mendis', 'M Theekshana', 'JDF Vandersay', 'K Mishara',
       'J Liyanage', 'P Jayawickrama', 'N Thushara', 'PM Liyanagamage',
       'D Madushanka', 'M P

In [4]:
# Get the data for BKG Mendis if batter is BKG Mendis or non-striker is BKG Mendis
bkg_mendis_data = data.loc[
    ((data['batter'] == 'BKG Mendis') | (data['non_striker'] == 'BKG Mendis')) & (data['innings'] < 3)
]

bkg_mendis_data.head()

,Unnamed: 0,batter,bowler,non_striker,runs_by_bat,extra_runs,total_runs_delivery,current_team_total,runs_remain,batter_runs,...,winning_team,batting_team,bowling_team,won,final_team_total,batter_type,non_striker_type,bowler_type,replacements,review
387,14,BKG Mendis,JP Faulkner,EMDY Munaweera,0,0,0,22,151.0,0,...,Sri Lanka,Sri Lanka,Australia,1.0,176,Right hand Bat,Right hand Bat,Left arm Fast,NaN,NaN
388,15,BKG Mendis,JP Faulkner,EMDY Munaweera,2,0,2,24,149.0,2,...,Sri Lanka,Sri Lanka,Australia,1.0,176,Right hand Bat,Right hand Bat,Left arm Fast,NaN,NaN
389,16,BKG Mendis,JP Faulkner,EMDY Munaweera,2,0,2,26,147.0,4,...,Sri Lanka,Sri Lanka,Australia,1.0,176,Right hand Bat,Right hand Bat,Left arm Fast,NaN,NaN
390,17,BKG Mendis,JP Faulkner,EMDY Munaweera,1,0,1,27,146.0,5,...,Sri Lanka,Sri Lanka,Australia,1.0,176,Right hand Bat,Right hand Bat,Left arm Fast,NaN,NaN
391,18,BKG Mendis,JA Richardson,EMDY Munaweera,0,0,0,27,146.0,5,...,Sri Lanka,Sri Lanka,Australia,1.0,176,Right hand Bat,Right hand Bat,Right arm Fast,NaN,NaN


In [97]:
# group data by match_id
gp = bkg_mendis_data.groupby('match_id')
cols = [ 'batter', 'non_striker', 'batter_runs', 'balls_faced', 'wicket_type', 'won', 'innings', 'over', 'delivery']
bkg_mendis_scores= gp.last().loc[:, cols]

# 3 matches missing from the data



# when BKG Mendis is the non-striker when the last ball was bowled
# The batter_runs and balls_faced are not his, but the on_strike batter's
# So, we need to get the last ball he faced
# he might not even have faced a ball

# get the last ball he faced

matches_non_striker = bkg_mendis_scores[bkg_mendis_scores['non_striker'] == 'BKG Mendis'].index
# get the last batter == BKG Mendis row from gp data
gp = bkg_mendis_data[bkg_mendis_data['batter'] == 'BKG Mendis'].groupby(['match_id'])

bkg_mendis_scores.loc[matches_non_striker, ['batter_runs', 'balls_faced']] = gp.last().loc[matches_non_striker][['batter_runs', 'balls_faced']]
bkg_mendis_scores

,batter,non_striker,batter_runs,balls_faced,wicket_type,won,innings,over,delivery
match_id,,,,,,,,,
Afghanistan_Sri Lanka_2022-09-03,BKG Mendis,P Nissanka,36,19,caught,1.0,2,6,3.0
Afghanistan_Sri Lanka_2022-11-01,BKG Mendis,DM de Silva,25,29,caught,1.0,2,7,5.0
Afghanistan_Sri Lanka_2024-02-21,BKG Mendis,P Nissanka,16,14,caught,0.0,2,5,5.0
Australia_Sri Lanka_2017-02-19,BKG Mendis,EMDY Munaweera,5,7,caught,1.0,2,3,3.0
Australia_Sri Lanka_2017-02-22,BKG Mendis,EMDY Munaweera,14,10,caught,0.0,2,7,3.0
...,...,...,...,...,...,...,...,...,...
Sri Lanka_West Indies_2020-03-04,BKG Mendis,MDKJ Perera,0,3,caught,0.0,2,1,6.0
Sri Lanka_West Indies_2020-03-06,BKG Mendis,AD Mathews,11,13,caught,0.0,1,11,2.0
Sri Lanka_Zimbabwe_2024-01-16,BKG Mendis,S Samarawickrama,4,5,caught,0.0,1,3,4.0


In [98]:
# adding new features
# strike rate
bkg_mendis_scores['strike_rate'] = round(bkg_mendis_scores['batter_runs'] / bkg_mendis_scores['balls_faced'] * 100, 2)
bkg_mendis_scores['out'] = bkg_mendis_scores['wicket_type'] != '0'